In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model = "gpt-4o-mini")

llm.invoke([HumanMessage("한가하니?")])

AIMessage(content='네, 저는 항상 여러분의 질문에 답할 준비가 되어 있습니다. 어떤 도움이 필요하신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 12, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1DeDoqVVL0IalgKgeogk3UX1xjAj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2f6d0af1-3128-46a6-844b-3ee9209a01c6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 23, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
from re import L
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone: str, location: str) -> str:
  """ 현재 시각을 반환하는 함수
  
  Args:
    timezone(str): 타임존(e.g. Asia/Seoul) 실제 존재해야 함
    location(str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
  """
  
  tz = pytz.timezone(timezone)
  now = datetime.now(tz)
  location_and_local_time = f"{timezone} {location}의 현재 시각: {now}"
  print(location_and_local_time)
  
  return location_and_local_time


In [11]:
tools = [get_current_time]
tool_dict = {
  "get_current_time": get_current_time
}

llm_with_tools = llm.bind_tools(tools)

In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
  SystemMessage("너는 사용자의 질문에 답변하기 위해 tools 를 사용할 수 있다."),
  HumanMessage("성남시의 현재 시각은 몇 시인가요?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변하기 위해 tools 를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='성남시의 현재 시각은 몇 시인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uBt7Z413wqkBfmXBuoVXhwOy', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"성남시"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 131, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1DeLruHvO3C97CAKPDw2nUbiLUyj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--85377bba-38bc-4cf0-888b-d5ba7b002c82-0', to

In [14]:
for tool_call in response.tool_calls:
  selected_tool = tool_dict[tool_call["name"]]
  print(tool_call["args"])
  
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)
  
messages

{'timezone': 'Asia/Seoul', 'location': '성남시'}
Asia/Seoul 성남시의 현재 시각: 2025-08-06 00:15:17.223082+09:00


[SystemMessage(content='너는 사용자의 질문에 답변하기 위해 tools 를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='성남시의 현재 시각은 몇 시인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uBt7Z413wqkBfmXBuoVXhwOy', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"성남시"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 131, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1DeLruHvO3C97CAKPDw2nUbiLUyj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--85377bba-38bc-4cf0-888b-d5ba7b002c82-0', 

In [15]:
llm_with_tools.invoke(messages)

AIMessage(content='성남시의 현재 시각은 2025년 8월 6일 00시 15분 17초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 196, 'total_tokens': 226, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1Dg0kN1xKkmTlqHOlQuFVtHYlB2D', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ff36074a-b4f2-41d8-9fc1-346ffdc01254-0', usage_metadata={'input_tokens': 196, 'output_tokens': 30, 'total_tokens': 226, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
  """ 주식 시장 데이터를 반환하는 함수
  
  Args:
    stock_history_input(StockHistoryInput): 주식 시장 데이터 입력
  """
  
  stock = yf.Ticker(stock_history_input.ticker)
  history = stock.history(period = stock_history_input.period)
  history_md = history.to_markdown()
  
  return history_md

tools = [
  get_current_time,
  get_yf_stock_history
]

tool_dict = {
  "get_current_time": get_current_time,
  "get_yf_stock_history": get_yf_stock_history
}

llm_with_tools = llm.bind_tools(tools)

messages = [
  SystemMessage("너는 사용자의 질문에 답변하기 위해 tools 를 사용할 수 있다."),
]

NameError: name 'StockHistoryInput' is not defined